In [1]:
# embedding : 기존 one-hot encoding에서는 비효율적인 0 값이 많음
# 즉, 대응되는 word의 위치만 1이고 나머지는 0
# 그러므로 encoding vector 간에 연관성 찾기가 어렵고, 표현 자체가 비효율적
# 따라서 embedding matrix를 이용해 one-hot encoding에 곱한 것을 embedding으로 통칭

# one-hot encoding : learning = [0, 0, 1, 0, 0]
# embedding : learning = [0.123, 0.456, -0.789]

# 대표적인 embedding 기법
# word2vec, fastEmbed, Bert, ...

# vector store (vector db) : embedding된 벡터들을 저장해놓은 db 
# vector db를 바탕으로 RAG에서 검색 시, query와 가장 유사도가 높은 vector를 뽑아
# 해당 내용을 검색함

ref : 

https://colab.research.google.com/drive/1hiCOynlzt7jJ_U5q6DqZQ3mJO7VZ_Em1?usp=sharing

https://python.langchain.com/docs/integrations/text_embedding/

아직 llama3에 대한 embedding이 langchain의 LlamaCppEmbeddings에 안 되어 있는 듯..??

=> llama2로 변경해서 진행해도 똑같은 에러 발생
(error : float() argument must be a string or a number, not 'list')

=> 코드 보니 client.embed 라는 메서드 호출 결과를 저장할 때 단순 list가 되어야 하는데 list(list) 로
구성되어 있어서 발생하는 듯

=> 하지만 여기는 건들 수 없는 영역....

llamafile로 변경 후 진행

=> windows에서 이상한 파일이라고 실행 막아버림

=> 레지스트리 설정 바꿔서 할 수 있을 거 같지만 굳이...?? 

Ollama로 변경 후 진행

=> 아직 windows 버전은 preview.... (불안정...)

=> model 저장 경로 : `C:\Users\<username>\.ollama\models`


In [2]:
# llama embedding 
from langchain_community.embeddings import OllamaEmbeddings

embeddings_model = OllamaEmbeddings(
    model="llama3:8b-instruct-q4_K_M",
)

In [3]:
docs = [
    "hello",
    "red ball",
    "blue ball",
    "green ball",
    "orange ball",
]

In [4]:
# document에 대한 임베딩 진행
embeddings = embeddings_model.embed_documents(docs)

print(embeddings)

[[0.7029784917831421, -1.1111974716186523, -0.3591102063655853, -1.6644188165664673, 1.335403561592102, 0.26233169436454773, 0.6234388947486877, -0.984709620475769, -0.7315595746040344, 0.6601229310035706, 0.29722654819488525, -1.2596559524536133, -1.3800541162490845, 0.9959756135940552, -1.7724614143371582, 3.865453004837036, -2.068603754043579, 0.2018853724002838, -0.6886011362075806, 0.27206963300704956, 1.8450433015823364, -0.017698321491479874, 0.917233943939209, 0.5855376720428467, -2.562251091003418, -0.1431703418493271, 2.2051024436950684, 1.235443115234375, -0.2820834219455719, -1.0577021837234497, -0.062203794717788696, 1.0501575469970703, -1.9832144975662231, 1.4573062658309937, -0.5620838403701782, -0.4810665249824524, -0.25516945123672485, -0.15552715957164764, -0.2578364312648773, -2.3638577461242676, -0.765847384929657, -1.1960026025772095, 2.804896354675293, -1.5687867403030396, 2.115368127822876, -0.6532549262046814, 1.6446571350097656, -2.686159133911133, 2.8441135883

In [5]:
# query에 대한 임베딩 진행 
# 실제 임베딩 모델에는 반영 X
embedded_query = embeddings_model.embed_query("ball")
print(embedded_query)

[-3.89562726020813, -1.8017758131027222, -0.7774577736854553, -1.9225736856460571, 1.1407619714736938, -0.5577744841575623, -1.2892228364944458, 0.8081857562065125, -3.3628242015838623, -3.4085335731506348, -1.2703566551208496, 0.8251743316650391, -2.4701972007751465, 1.255049467086792, -1.0528814792633057, 1.9530134201049805, -1.221995234489441, -0.9407762885093689, -2.69286847114563, -1.2156349420547485, -0.7854437828063965, -1.1176878213882446, 3.8843307495117188, -1.6883232593536377, -0.1965043693780899, -0.8749687671661377, 2.0596795082092285, 2.7075648307800293, 2.578827381134033, 1.3927075862884521, -0.783166766166687, 1.298362135887146, 1.409500002861023, -1.7894752025604248, 0.6021122932434082, -1.9023780822753906, 1.2392362356185913, 0.06862588971853256, 0.26875001192092896, -0.5400657057762146, 1.0722169876098633, -0.2605077028274536, -1.2037936449050903, 1.66517174243927, -1.1060333251953125, -1.7647639513015747, 1.6487135887145996, -2.3971986770629883, -2.177860736846924, 

In [15]:
# vector db 구성
# chroma db 이용 

from langchain.docstore.document import Document

sample_texts = [
    "hello",
    "red",
    "blue",
    "green",
    "orange",
]

In [16]:
documents = []

for text in sample_texts : 
    page = Document(page_content=text)
    documents.append(page)

In [17]:
documents

[Document(page_content='hello'),
 Document(page_content='red'),
 Document(page_content='blue'),
 Document(page_content='green'),
 Document(page_content='orange')]

In [18]:
# similarity search 

from langchain.vectorstores import Chroma

# document 내용을 임베딩 모델에 적용
db = Chroma.from_documents(documents, embeddings_model)

In [24]:
# 쿼리 내용에 대해 가장 유사한 벡터 찾아서 리턴
query1 = "red"
doc1 = db.similarity_search(query1)

print(doc1)

query2 = "red ball"
doc2 = db.similarity_search(query2)

print(doc2)

[Document(page_content='red'), Document(page_content='blue'), Document(page_content='green'), Document(page_content='orange')]
[Document(page_content='red ball'), Document(page_content='green ball'), Document(page_content='blue ball'), Document(page_content='orange ball')]


In [27]:
# vector 기준으로 search

query3 = "blue blue"
embedding_vector = embeddings_model.embed_query(query3)
doc3 = db.similarity_search_by_vector(embedding_vector)

print(doc3)

[Document(page_content='blue'), Document(page_content='green'), Document(page_content='hello'), Document(page_content='hello')]
